In [1]:
from utils import save_stock_prices_nyse
import pandas as pd
import datetime as dt
import numpy as np
from threading import Thread
from queue import Queue
import time
import os

In [2]:
start_date = dt.datetime(2009, 1, 1)
end_date = dt.datetime(2016, 11, 1)
# symbols = ['aapl', 'mmm', 'aaa', 'btz', 'bud', 'bwpt', 'bx', 'eot','au','auo']
folder_stock_prices = './test'
folder_stock_symbols = './stock_symbols'

In [3]:
params_ = Queue()

In [4]:
df_nyse_symbols = pd.read_csv(os.path.join(folder_stock_symbols, 'NYSE.csv'))
df_nyse_symbols.columns = [x.replace(' ', '_').lower() for x in df_nyse_symbols.columns]
df_nyse_symbols['symbol_lower'] = df_nyse_symbols['symbol'].str.lower()
df_nyse_symbols['ticker'] = 'NYSE:' + df_nyse_symbols['symbol']

In [5]:
def create_queue():
    ticker_have = [i.strip('nyse_').replace('.csv', '') for i in os.listdir(folder_stock_prices) if '.DS' not in i]
    df_temp =  df_nyse_symbols.loc[~df_nyse_symbols['symbol_lower'].isin(ticker_have)]
    symbols = df_temp['symbol_lower'].tolist()
    for symbol in symbols:
        params_.put((folder_stock_prices, symbol, start_date, end_date))

In [6]:
def main():
    while True:
        p = params_.get()
        save_stock_prices_nyse(*p)
        params_.task_done()

In [ ]:
create_queue()
t1 = time.time()
number_of_threads = 10
for _ in range(number_of_threads):
    worker = Thread(target=main, daemon=True)
    worker.start()
params_.join()
t2 = time.time()
print(t2 - t1)

Exception in thread Thread-9:
Traceback (most recent call last):
  File "/Users/chadupjohn/.virtualenvs/personal/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py", line 595, in urlopen
    chunked=chunked)
  File "/Users/chadupjohn/.virtualenvs/personal/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py", line 393, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/Users/chadupjohn/.virtualenvs/personal/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py", line 389, in _make_request
    httplib_response = conn.getresponse()
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/http/client.py", line 1197, in getresponse
    response.begin()
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python

In [6]:
from time import sleep
import sys

def radprint(string):
    for char in string:
        sys.stdout.write(char)
        sys.stdout.flush()
        sleep(0.05)
    sys.stdout.write('\n')

In [11]:
import asyncio

import aiohttp
import time

async def speak(animal, session):
    """
    Retrieves the sound for the given animal,
    and prints it with animation.
    """
    print(animal, '1', time.time())
    response = await session.get(
        'https://ericappelt.com/animals/{0}'.format(animal)
    )
    print(animal, '2', time.time())
    sound = await response.text()
    radprint('The {0} says "{1}".'.format(animal, sound))


async def main():
    """
    Retrieve and print sounds for all animals.
    """
    animals = ['cow', 'pig', 'chicken']
    tasks = []
    async with aiohttp.ClientSession() as session:
        for animal in animals:
            print('yo', time.time())
            fut = asyncio.ensure_future(speak(animal, session))
            tasks.append(fut)

        await asyncio.wait(tasks)


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

yo 1481480003.683805
yo 1481480003.68388
yo 1481480003.683909
cow 1 1481480003.683977
pig 1 1481480003.685299
chicken 1 1481480003.686149
cow 2 1481480011.31447
The cow says "Moo!".
pig 2 1481480012.383408
The pig says "Oink!".
chicken 2 1481480013.503361
The chicken says "Cluck!".
